# Modeling

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plot
%matplotlib inline

### Datasets

In [2]:
clean_df = pd.read_csv('../data/clean/tanzania.csv', index_col=0)

### Drop Lat and Long

In [3]:
X = clean_df.drop(['target', 'latitude', 'longitude', 'population'], axis=1)
y = clean_df['target']

In [4]:
numerical_cols = list(X.select_dtypes(include=np.number).columns)
categorical_cols = list(X.select_dtypes(exclude=np.number).columns)

### Creating Pipeline

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scale', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

## Models
- LinearSVC
- k-NN
- Support Vector Machine Algorithm
- XGBoost
- Random Forest

### Predict
TODO: add SMOTE option

In [6]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

def fit_predict(model, X_train, X_test, y_train, y_test):
    '''fit pipeline using given model, and return predictions'''
    
    param_grid = model['params']
    model = model['model']
        
    # Bundle preprocessing and modeling code in a pipeline
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('model', model)
                                 ])
    
    search = GridSearchCV(estimator=my_pipeline,
             param_grid=param_grid, n_jobs=-1)
    
    search.fit(X_train, y_train)
    
    print("Best parameter (CV score=%0.3f):" % search.best_score_)
    print(search.best_params_)

    # Preprocessing of validation data, get predictions
    test_preds = search.predict(X_test)
    train_preds = search.predict(X_train)
    return test_preds, train_preds

## Evaluate
What are the most important things to look for with regression?

TODO: add graphs

In [7]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

def metrics(model_name, y_train, y_test, y_train_pred, y_test_pred):
    '''Print out the evaluation metrics for a given models predictions'''
    print(f'Model: {model_name}', )
    print('-'*60)
    print(f'test accuracy: {accuracy_score(y_test, y_test_pred)}')
    print(f'train accuracy: {accuracy_score(y_train, y_train_pred)}')
    print('-'*60)
    print('\ntest report:\n' + classification_report(y_test, y_test_pred))
    print('~'*60)
    print('\ntrain report:\n' + classification_report(y_train, y_train_pred))    
    print('-'*60)


## LinearSVC

### Why LinearSVC
The objective of a Linear SVC (Support Vector Classifier) is to fit to the data you provide, returning a "best fit"
hyperplane that divides, or categorizes, your data. 

It has more flexibility in the choice of penalties and loss functions and should scale better to large numbers of samples.

LinearSVC is another (faster) implementation of Support Vector Classification for the case of a linear kernel.

### Hyperparameters

In [8]:
param_dict = {
    # 
    'model__C': [0.1,1, 10, 100], 
    'model__tol': [1,0.1,0.01,0.001],
    'model__max_iter': [10000],
    'model__penalty': ['l1', 'l2'],
    'model__multi_class': ['ovr', 'crammer_singer']
}

### Fitting LinearSVC

In [ ]:
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

lsvc = { 'model': LinearSVC(), 'params': param_dict }

test_preds, train_preds = fit_predict(lsvc, X_train, X_test, y_train, y_test)

### Evaluating LinearSVC

In [ ]:
metrics('LinearSVC', y_train, y_test, train_preds, test_preds)

There is a debate that LinearSVC is not a good fit for multi-class classification problems.

There are some ways to adjust for it: 
- using the 'crammer_singer' algorithm

## KNN

### Why KNN
KNNs are a non-parametric, lazy learning algorithm. It makes no underlying assumptions about the distribution of data. 

No training is necessary! 

KNN makes predictions just-in-time by calculating the similarity between an input sample and each training instance.

It is a Simple algorithm — to explain and understand/interpret. It is versatile — useful for classification or regression.

### Hyperparameters

In [ ]:
param_dict = {
    # amount of neighbors
    'model__n_neighbors': range(1, 10, 2),
    # leaf size
    'model__leaf_size': range(30, ),
    'model__weights': ['uniform', 'distance']
}

### Fitting KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = { 'model': KNeighborsClassifier(), 'params': param_dict }

test_preds, train_preds = fit_predict(knn, X_train, X_test, y_train, y_test)

### Evaluating KNN

In [ ]:
metrics('k-NN', y_train, y_test, train_preds, test_preds)